In [1]:
import tensorflow

In [2]:
import keras

Using TensorFlow backend.


In [3]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [4]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 663866317148413193
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285974221
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2615025874808307667
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [5]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [40]:
img_width, img_height = 128, 128
train_data_dir = "/data/train_binary"
validation_data_dir = "/data/train_binary"
nb_train_samples = 2942
nb_validation_samples = 326 
batch_size = 512
epochs = 50

In [7]:
nr_categories = 2

In [8]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_1.h5")

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [10]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [11]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [12]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [13]:
# Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model_final.layers[:12]:
    print(layer.name)
    layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool


In [14]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [15]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [16]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [17]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [41]:
img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 2942 images belonging to 2 classes.


In [42]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 326 images belonging to 2 classes.


In [30]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_binary_cat200_cat300_128_p2_b.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')


In [21]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
#batch=8

In [26]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
368/367 [==============================] - 86s 233ms/step - loss: 0.0201 - acc: 0.9915 - val_loss: 0.1064 - val_acc: 0.9755

Epoch 00001: val_acc did not improve from 0.98160
Epoch 2/50
368/367 [==============================] - 83s 227ms/step - loss: 0.0176 - acc: 0.9932 - val_loss: 0.0722 - val_acc: 0.9908

Epoch 00002: val_acc improved from 0.98160 to 0.99080, saving model to vgg16_binary_cat200_cat300_128_p2_b.h5
Epoch 3/50
368/367 [==============================] - 83s 227ms/step - loss: 0.0060 - acc: 0.9976 - val_loss: 0.1290 - val_acc: 0.9663

Epoch 00003: val_acc did not improve from 0.99080
Epoch 4/50
368/367 [==============================] - 84s 227ms/step - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0844 - val_acc: 0.9877

Epoch 00004: val_acc did not improve from 0.99080
Epoch 5/50
368/367 [==============================] - 84s 227ms/step - loss: 0.0048 - acc: 0.9983 - val_loss: 0.0932 - val_acc: 0.9847

Epoch 00005: val_acc did not improve from 0.99080
Epoch 6/50

In [ ]:
#batch = 64

In [31]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
46/45 [==============================] - 84s 2s/step - loss: 4.0018e-04 - acc: 1.0000 - val_loss: 0.1061 - val_acc: 0.9847

Epoch 00001: val_acc improved from -inf to 0.98466, saving model to vgg16_binary_cat200_cat300_128_p2_b.h5
Epoch 2/50
46/45 [==============================] - 74s 2s/step - loss: 8.1595e-04 - acc: 0.9997 - val_loss: 0.1035 - val_acc: 0.9847

Epoch 00002: val_acc did not improve from 0.98466
Epoch 3/50
46/45 [==============================] - 73s 2s/step - loss: 5.4017e-04 - acc: 1.0000 - val_loss: 0.1053 - val_acc: 0.9847

Epoch 00003: val_acc did not improve from 0.98466
Epoch 4/50
46/45 [==============================] - 73s 2s/step - loss: 2.5855e-04 - acc: 1.0000 - val_loss: 0.1063 - val_acc: 0.9847

Epoch 00004: val_acc did not improve from 0.98466
Epoch 00004: early stopping


In [ ]:
#batch = 128

In [35]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
23/22 [==============================] - 87s 4s/step - loss: 2.4230e-04 - acc: 1.0000 - val_loss: 0.1066 - val_acc: 0.9847

Epoch 00001: val_acc improved from 0.98466 to 0.98466, saving model to vgg16_binary_cat200_cat300_128_p2_b.h5
Epoch 2/50
 8/22 [=========>....................] - ETA: 29s - loss: 2.0221e-04 - acc: 1.0000

KeyboardInterrupt: 

In [ ]:
#batch = 256

In [39]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
12/11 [===============================] - 85s 7s/step - loss: 3.3047e-04 - acc: 1.0000 - val_loss: 0.1067 - val_acc: 0.9847

Epoch 00001: val_acc did not improve from 0.98466
Epoch 2/50
12/11 [===============================] - 78s 7s/step - loss: 2.5616e-04 - acc: 1.0000 - val_loss: 0.1069 - val_acc: 0.9847

Epoch 00002: val_acc did not improve from 0.98466
Epoch 3/50
 3/11 [======>.......................] - ETA: 29s - loss: 6.8288e-04 - acc: 1.0000

KeyboardInterrupt: 

In [ ]:
#batch = 512

In [43]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
6/5 [===============================] - 107s 18s/step - loss: 4.0530e-04 - acc: 1.0000 - val_loss: 0.1076 - val_acc: 0.9847

Epoch 00001: val_acc improved from 0.98466 to 0.98466, saving model to vgg16_binary_cat200_cat300_128_p2_b.h5
Epoch 2/50
1/5 [====>.........................] - ETA: 12s - loss: 8.1740e-05 - acc: 1.0000

KeyboardInterrupt: 

In [ ]:
!ls

In [ ]:
batch_size = 16

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/10),
        class_mode=None) #,  # this means our generator will only yield batches of data, no labels
#) #shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, 60000)
# save the output as a Numpy array
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/20),
        class_mode=None)
#) #shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 8000)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nr_categories, activation='softmax')) 
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

In [ ]:
print(1)